# 🎶 [Day 16](https://adventofcode.com/2019/day/16)

In [1]:
def one_phase_fft(inputs, base_pattern=(0, 1, 0, -1)):
    """A naive implementation"""
    outputs = [0] * len(inputs)
    for i in range(len(inputs)):
        res = 0
        j = 0
        try:
            while 1:
                for b in base_pattern:
                    for repeat in range(i + 1):
                        # skip first
                        if j == 0:
                            j += 1
                            continue
                        # Add to result
                        res += inputs[j - 1] * b
                        j += 1
                        # Break when reaching end of inputs
                        if j == len(inputs) + 1:
                            raise ValueError
        except ValueError:
            pass
        outputs[i] = abs(res) % 10
    return outputs


def one_phase_fft_efficient(inputs, base_pattern=(0, 1, 0, -1)):
    """A more efficient implementation, by skipping zero entries altogether"""
    outputs = [0] * len(inputs)
    for pos in range(len(inputs)):
        res = 0
        modulo = pos % len(inputs) + 1
        i = modulo
        # First pattern (+ skip first)
        while i <= len(inputs):
            res += sum(inputs[i - 1:i + modulo - 1])
            res -= sum(inputs[i + 2 * modulo - 1: i + 3 * modulo - 1])
            i += 4 * modulo
        outputs[modulo - 1] = abs(res) % 10
    return outputs


def fft(inputs, num_phases, fft_fn=one_phase_fft, repeat_inputs=1):
    """If inputs repeat we can leverage a recurrence relationship:
    
    If inputs has length n, and if we note S the result of fft on `inputs`,
    and T the results of fft on `inputs + inputs` then we have:
    T[n + ceil(n/2)] = S[ceil(n / 2)] which are just the sum of 
    """
    outputs = [x for x in inputs]
    for _ in range(num_phases):
        outputs = fft_fn(outputs)
    return ''.join(list(map(str, outputs)))


import numpy as np
def part2_fft(input_list, num_phases, num_repeats=10000):
    """Looking at the previous examples, we can see that whenever the offset is large 
    and the "-1" part of the pattern doesn't appear, then the output of FFT is 
    just a cumulative sum: For input x, for all index i > len(x) / 2, we have:
        FFT(i) = sum(j=i to len(x))
    Hence computing coefficients at the end of the list only requires knowing 
    inputs towards the end of the list too.
    
    We can use this trick in part 2 because the numbers we care about (offset
    given by the first 7 numbers in the input) are towards the end of the list
    """
    inputs = input_list[::-1]
    offset = sum([v * 10**i for i, v in enumerate(inputs[-7:])])
    if offset <= len(inputs) * num_repeats / 2:
        print("Cannot use this trick use standard fft")
        return

    # Compute only on inputs we will use
    repeated = int(np.ceil(offset // len(inputs)))
    precompute = inputs * (num_repeats - repeated - 1) + inputs[:-offset % len(inputs)]
    
    # Apply cumsum repeatedly
    for _ in range(num_phases):
        precompute = np.mod(np.cumsum(precompute), 10)
    # Return 8 numbers
    return sum([v * 10**i for i, v in enumerate(precompute[-8:])])

In [2]:
%%time
with open("inputs/day16.txt", 'r') as f:
    inputs = list(map(int, f.read().splitlines()[0]))
    
print("The first 8 digits of the fft are {}\n".format(fft(inputs, 100)[:8]))

The first 8 digits of the fft are 19944447

CPU times: user 12.3 s, sys: 34 ms, total: 12.3 s
Wall time: 12.5 s


In [3]:
%%time
print("Comparing timing with efficient implementation (returns {})\n".format(
    fft(inputs, 100, fft_fn=one_phase_fft_efficient)[:8]))

Comparing timing with efficient implementation (returns 19944447)

CPU times: user 277 ms, sys: 1 µs, total: 277 ms
Wall time: 279 ms


In [5]:
%%time
print("The result for Part 2 is {}\n".format(part2_fft(inputs, 100)))

The result for Part 2 is 81207421

CPU times: user 816 ms, sys: 16 ms, total: 832 ms
Wall time: 831 ms
